# Comparaison de la performance des trois types de modèles

On compare ici la performance d'une classifieur sans contrainte contre un DLP et un classifieur avec des seuils de décision disparates pour chaque groupe.

On vérifie à chaque fois:
- l'accuracy globale du modèle
- Le respect de la parité démographique (contrainte sur laquelle on optimise)
- Le respect de la règle p%

In [1]:
# On place l'exécution du code à la racine du projet.
import sys
from pathlib import Path

root_path = Path().resolve().parent  
sys.path.append(str(root_path))

In [3]:
import pandas as pd
from data.preprocessing import prepare_data

X_train, X_test, y_train, y_test, protected_train, protected_test = prepare_data()

## Régression logistique standard

In [4]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

Calcul des métriques importantes: accuracy, demographic parity, p%-rule

In [5]:
from sklearn.metrics import classification_report

report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.93      0.90     11543
           1       0.73      0.60      0.66      3772

    accuracy                           0.85     15315
   macro avg       0.80      0.77      0.78     15315
weighted avg       0.84      0.85      0.84     15315



In [16]:
def print_metric_result(result):
    print(f'value: {result[0]:.2f} \n' \
          f'Most advantaged_group: {result[1][0]} \n' \
          f'Most disadvantaged_group: {result[1][1]}')

In [18]:
from fairness.fair_metrics import intersectional_demographic_parity, intersectional_p_percent
idp = intersectional_demographic_parity(y_pred,protected_test)
print('Demographic Parity:')
print_metric_result(idp)

Demographic Parity:
value: 0.18 
Most advantaged_group: Asian-Pac-Islander 
Most disadvantaged_group: Amer-Indian-Eskimo


In [20]:
p_per = intersectional_p_percent(y_pred,protected_test)
print('P%-Rule')
print_metric_result(p_per)

P%-Rule
value: 0.32 
Most advantaged_group: Asian-Pac-Islander 
Most disadvantaged_group: Amer-Indian-Eskimo


## DLP avec Fairlearn

Fairlearn propose nativement une implémentation d'un Disparate Learning Process par pénalisation de la fonction de perte par une contraite. On choisit ici la parité démorgaphique.

In [22]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

constraint = DemographicParity()

clf_fair = ExponentiatedGradient(
    estimator=LogisticRegression(solver="liblinear"),
    constraints=constraint,
    eps=0.02  # tolérance fairness (à explorer)
)

clf_fair.fit(
    X_train,
    y_train,
    sensitive_features=protected_train
)

y_pred_fair = clf_fair.predict(X_test)

Les mêmes métriques sont calculés pour comparaison

In [23]:
fair_report = classification_report(y_test,y_pred_fair)
print(fair_report)

              precision    recall  f1-score   support

           0       0.81      0.92      0.86     11543
           1       0.57      0.32      0.41      3772

    accuracy                           0.77     15315
   macro avg       0.69      0.62      0.63     15315
weighted avg       0.75      0.77      0.75     15315



In [27]:
fair_idp = intersectional_demographic_parity(y_pred_fair,protected_test)
print('Demographic Parity (fair):')
print_metric_result(fair_idp)

Demographic Parity (fair):
value: 0.03 
Most advantaged_group: Amer-Indian-Eskimo 
Most disadvantaged_group: Other


In [28]:
fair_p_per = intersectional_p_percent(y_pred_fair,protected_test)
print('P%-rule (fair):')
print_metric_result(fair_p_per)

P%-rule (fair):
value: 0.80 
Most advantaged_group: Amer-Indian-Eskimo 
Most disadvantaged_group: Other


## Treatment disparity par seuillage

In [31]:
from fairness.treatment_disparity import MulticlassThresholdOptimizer

best_fair_clf = MulticlassThresholdOptimizer(protected_test)

opti_y_pred = best_fair_clf.fit_transform(X_train,y_train,X_test,gamma=0.001)

In [32]:
report = classification_report(y_test,opti_y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.82      0.86     11543
           1       0.57      0.75      0.65      3772

    accuracy                           0.80     15315
   macro avg       0.74      0.78      0.75     15315
weighted avg       0.83      0.80      0.81     15315



In [33]:
opti_fair_idp = intersectional_demographic_parity(opti_y_pred,protected_test)
print('Difference in Demographic Parity (optimal fair):')
print_metric_result(opti_fair_idp)

Difference in Demographic Parity (optimal fair):
value: 0.00 
Most advantaged_group: Asian-Pac-Islander 
Most disadvantaged_group: White


In [34]:
opti_fair_p_per = intersectional_p_percent(opti_y_pred,protected_test)
print('P%-rule (optimal fair):')
print_metric_result(opti_fair_p_per)

P%-rule (optimal fair):
value: 1.00 
Most advantaged_group: Asian-Pac-Islander 
Most disadvantaged_group: White
